<a href="https://colab.research.google.com/github/SallyPeter/gomycodeDSbootcamp/blob/main/ML/Machine_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This data was collected from a Kaggle competition with the goal of predicting whether a customer continues with their account or closes it (e.g., churns)**

***This is a Classification Task***


In [ ]:
# Importing Libraries

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# !pip install kaggle -q;

# import kaggle
# kaggle competitions download -c ai-ml-for-all

Reading in the data and having an overview of it

In [ ]:
# Load dataset
data = pd.read_csv("train.csv")

data.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               165034 non-null  int64  
 1   CustomerId       165034 non-null  int64  
 2   Surname          165034 non-null  object 
 3   CreditScore      165034 non-null  int64  
 4   Geography        165034 non-null  object 
 5   Gender           165034 non-null  object 
 6   Age              165034 non-null  float64
 7   Tenure           165034 non-null  int64  
 8   Balance          165034 non-null  float64
 9   NumOfProducts    165034 non-null  int64  
 10  HasCrCard        165034 non-null  float64
 11  IsActiveMember   165034 non-null  float64
 12  EstimatedSalary  165034 non-null  float64
 13  Exited           165034 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 17.6+ MB


In [ ]:
data.describe()

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,165034.0000,1.650340e+05,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000
mean,82516.5000,1.569201e+07,656.454373,38.125888,5.020353,55478.086689,1.554455,0.753954,0.497770,112574.822734,0.211599
std,47641.3565,7.139782e+04,80.103340,8.867205,2.806159,62817.663278,0.547154,0.430707,0.499997,50292.865585,0.408443
min,0.0000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,41258.2500,1.563314e+07,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74637.570000,0.000000
50%,82516.5000,1.569017e+07,659.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117948.000000,0.000000
75%,123774.7500,1.575682e+07,710.000000,42.000000,7.000000,119939.517500,2.000000,1.000000,1.000000,155152.467500,0.000000
max,165033.0000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


In [ ]:
data.Exited.value_counts()

,count
Exited,
0,130113
1,34921


The data is clean and does not need much cleaning and transformation. The classes are imbalanced with class 0 having over 130,000 records and class 1 having roughly 34,000 records.

In [ ]:
data.drop(['id', 'CustomerId', 'Surname'], axis=1, inplace=True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


**Encoding the categorical labels and storing the encoded classes in a dictionary to be reused during test and prediction on new data.**

In [ ]:
le = LabelEncoder()

# cat_cols = data['Geography','Gender']

le_order = {}
for each in ['Geography','Gender']:
  if each != 'uniqueid':
    data[each] = le.fit_transform(data[each])
    le_order[each] = list(le.classes_)

**Splitting the data into test and train**

In [ ]:
# Separate features and target
X = data.drop(['Exited'], axis=1)
y = data['Exited']

# smote = SMOTE(random_state=42)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# X_train, y_train = smote.fit_resample(X_train, y_train)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


Train shape: (148530, 10), Test shape: (16504, 10)


**Scaling the data to ensure the data is within same ranges**

In [ ]:
scaler = StandardScaler()
numerical_columns = ['CreditScore',  'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
# data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


**Initial model training**

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluate on the original test data
y_pred = model.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91     13012
           1       0.72      0.54      0.62      3492

    accuracy                           0.86     16504
   macro avg       0.80      0.74      0.77     16504
weighted avg       0.85      0.86      0.85     16504



In [ ]:
roc_auc_score(y_test, y_pred)

0.7425885665852896

In [ ]:
y_pred_1 = (model.predict_proba(X_test_scaled)[:,1] >= 0.3).astype(bool)
print(classification_report(y_test, y_pred_1))
print(roc_auc_score(y_test, y_pred_1))

              precision    recall  f1-score   support

           0       0.92      0.86      0.89     13012
           1       0.58      0.72      0.64      3492

    accuracy                           0.83     16504
   macro avg       0.75      0.79      0.77     16504
weighted avg       0.85      0.83      0.84     16504

0.7903728129713026


In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring="roc_auc")
scores

array([0.87197421, 0.87224813, 0.8723148 , 0.87692471, 0.87275671])

In [ ]:
y_train.value_counts()

,count
Exited,
0,117101
1,31429


In [ ]:
data.Exited.value_counts()

,count
Exited,
0,130113
1,34921


In [ ]:
y_pred_n = (model.predict_proba(X_test_scaled)[:,1] >= 0.3).astype(bool)
print(classification_report(y_test, y_pred_n))
print(roc_auc_score(y_test, y_pred_n))

              precision    recall  f1-score   support

           0       0.92      0.86      0.89     13012
           1       0.58      0.72      0.64      3492

    accuracy                           0.83     16504
   macro avg       0.75      0.79      0.77     16504
weighted avg       0.85      0.83      0.84     16504

0.7903728129713026


Running a random search on several values for the hyperparameters to pick the best model configuration for the training data.

In [ ]:
#Random Search

param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'class_weight': ['balanced', 'balanced_subsample', {0: 1, 1: 3}]
}

rf = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    scoring='roc_auc',
    n_iter=20,
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

best_rs = random_search.fit(X_train_scaled, y_train)


# Best parameters and performance
print("Best Parameters:", random_search.best_params_)
print("Best ROC AUC:", random_search.best_score_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 10, 'class_weight': {0: 1, 1: 3}}
Best ROC AUC: 0.8867753717947543


**The grid search was to test for all combination options in the hyperparameter grid but this was taking long hence not used.**

In [ ]:
# Grid Search

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'class_weight': ['balanced', 'balanced_subsample', {0: 1, 1: 3}]
}

rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    verbose=2,
    n_jobs=-1
)
grid_search.fit(X_train_scaled, y_train)

best_gs = grid_search.fit(X_train_scaled, y_train)


# Best parameters and performance
print("Best Parameters:", grid_search.best_params_)
print("Best ROC AUC:", grid_search.best_score_)

Fitting 3 folds for each of 243 candidates, totalling 729 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
best_rs = grid_search.fit(X_train_scaled, y_train)


# Best parameters and performance
print("Best Parameters:", grid_search.best_params_)
print("Best ROC AUC:", grid_search.best_score_)

**Using the best values from the random values**

In [ ]:
opt_model = RandomForestClassifier(n_estimators= 300, min_samples_split= 2, min_samples_leaf= 2, max_depth= 10, class_weight= {0: 1, 1: 3}, random_state=42) #, max_features= 'log2',
opt_model.fit(X_train_scaled, y_train)

# Evaluate on test data
y_pred = opt_model.predict(X_test_scaled)
print(classification_report(y_test, y_pred))
print(roc_auc_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.86      0.89     13012
           1       0.59      0.74      0.66      3492

    accuracy                           0.84     16504
   macro avg       0.76      0.80      0.78     16504
weighted avg       0.85      0.84      0.84     16504

0.8020928958342797


In [ ]:
# y_pred2 = (best_rs.predict_proba(X_test_scaled)[:,1] >= 0.3).astype(bool)

# pd.DataFrame(best_rs.predict_proba(X_test_scaled))[:15]

In [ ]:
# pd.DataFrame(best_rs.predict(X_test_scaled))[:15]

In [ ]:
# y_pred1 = best_rs.predict(X_test_scaled)

# print(classification_report(y_test, y_pred1))
# print(roc_auc_score(y_test, y_pred1))

**Predicting on test data**

In [ ]:
# Predicting on new data

test_data = pd.read_csv("test.csv")
test_data.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


In [ ]:
# test_data.columns
X_train.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'],
      dtype='object')

In [ ]:
data.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

**Imported the test data, fitting the encoder used in training to the test data**

In [ ]:

for col in le_order.keys():
  test_data[col] = test_data[col].map({x:le_order[col].index(x) for x in le_order[col]})


test_data[[ 'CreditScore', 'Geography', 'Gender','Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']] = scaler.transform(test_data[[ 'CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']])

test_data.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,-0.880139,-0.794904,-1.138260,-1.705266,-1.076971,-0.882727,0.814394,-1.754112,1.005631,0.963218
1,165035,15782418,Nott,0.331084,-0.794904,-1.138260,0.888849,-1.076971,-0.882727,-1.012522,0.570089,-0.994401,-0.794583
2,165036,15807120,K?,-0.006061,-0.794904,-1.138260,-0.464602,0.705357,-0.882727,0.814394,0.570089,-0.994401,0.524010
3,165037,15808905,O'Donnell,0.306110,-0.794904,0.878534,-0.239027,1.061822,-0.882727,-1.012522,0.570089,-0.994401,0.028032
4,165038,15607314,Higgins,1.192676,0.428615,0.878534,-0.013452,1.774753,1.048305,-1.012522,0.570089,-0.994401,0.534921


In [ ]:
# X_train_scaled

**Used the best model to predict the Exited feature on the test data**

In [ ]:
pred_outcome = opt_model.predict_proba(test_data.drop(["id", "CustomerId","Surname"], axis=1))

pred_outcome = pd.DataFrame(pred_outcome[:,1], columns=["Exited"])
pred_outcome

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


,Exited
0,0.061692
1,0.940521
2,0.104151
3,0.463983
4,0.567978
...,...
110018,0.166026
110019,0.222255
110020,0.066678
110021,0.375899


**Creating a dataframe with the predicted outcome and reading it to a CSV file.**

In [ ]:
test_data['Exited'] = pred_outcome


test_data[['id','Exited']].to_csv("My_Submission.csv", index=False)

In [ ]:
test_data[['id','Exited']].to_csv("My_Submission.csv", index=False)